In [1]:
import pandas as pd
import numpy as np
import inflect

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

# reload(sys)
# sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

import ExtractHelper as eh
import time

In [2]:
?eh.unpack_dictionary()

In [3]:
#code to put data from mongodb named Poetry
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY
poems = db.POEM
poemsdf = pd.DataFrame(list(poems.find()))

## some poem videos made it in the data so we need to remove them

In [5]:
poemsdf.columns
coltouse = ['poem_link', 'poet_poems_url','poem_yrpub','poem_title','poem_text']
# poemdf = rawpoem.loc[rawpoem['poem_title'] ][coltouse]

poemsdf = poemsdf.loc[poemsdf['poem_link'].str.contains('video') == False]
poemsdf = poemsdf[coltouse]

In [6]:
poemsdf.head()

,poem_link,poet_poems_url,poem_yrpub,poem_title,poem_text
1,http://www.poets.org/poetsorg/poem/te-deum,http://www.poets.org/poetsorg/poems/45091,1976,Te Deum,"[\nNot because of victories\nI sing,\nhaving n..."
2,http://www.poets.org/poetsorg/poem/notes-sprin...,http://www.poets.org/poetsorg/poems/45091,1976,"Notes on the Spring Holidays, III, [Hanukkah]","[\nIII\n, Hanukkah, \n\nIn a world where each ..."
3,http://www.poets.org/poetsorg/poem/coda-0,http://www.poets.org/poetsorg/poems/45669,1916,Coda,"[O my songs,, Why do you look so eagerly and s..."
4,http://www.poets.org/poetsorg/poem/my-philosop...,http://www.poets.org/poetsorg/poems/45448,1995,My Philosophy of Life,[\nJust when I thought there wasn't room enoug...
5,http://www.poets.org/poetsorg/poem/night-states,http://www.poets.org/poetsorg/poems/44618,2006,At Night the States,[\nAt night the states\nI forget them or I wis...


In [21]:
#compute the feature variables, number of lines, and number of stanzas
poemsdf['numlines'] = poemsdf['poem_text'].apply(lambda x: eh.countlines(x))
poemsdf['numstanzas'] = poemsdf['poem_text'].apply(lambda x: eh.countstanza(x))

In [23]:
#just a quick look
# poemdf.numlines
poemsdf.groupby(['poet_poems_url'])['numstanzas'].mean()[:5]

poet_poems_url
http://www.poets.org/poetsorg/poems/120970    1
http://www.poets.org/poetsorg/poems/120979    1
http://www.poets.org/poetsorg/poems/120986    1
http://www.poets.org/poetsorg/poems/121506    1
http://www.poets.org/poetsorg/poems/123231    1
Name: numstanzas, dtype: float64

In [25]:
#create feature variable that looks at the average number of lines per stanza for each poem_text
poemsdf['avgline_stanza'] = poemsdf['numlines']/poemsdf['numstanzas']

In [26]:
?eh.count_words()          

In [27]:
#create feature variables, wordcount = total number of words, wordcount_d = number of distinct words
#create feature variable ttr = type token ratio as a ratio wordcount_d:wordcount
poemsdf['wordcount'] = poemsdf['poem_text'].apply(lambda x: eh.count_words(x))
poemsdf['wordcount_d'] = poemsdf['poem_text'].apply(lambda x: eh.count_words(x,True))
poemsdf['ttr'] = poemsdf['wordcount_d']/poemsdf['wordcount']

In [28]:
?eh.grab_numerical()

In [29]:
#create feature variable - the year published for each poem
poemsdf['yrpub'] = poemsdf['poem_yrpub'].map(lambda x: eh.grab_numerical(x))

In [36]:
?eh.sent_tokenize()

In [37]:
#create a column of just tokenized sentences from the poem text
poemsdf['sentences'] = poemsdf['poem_text'].map(lambda x: eh.sent_tokenize(u' '.join(x)))

In [40]:
#check
# poemsdf.sentences.apply(lambda x: len(x))
# print poemsdf.sentences[12]

In [41]:
#create feature variable for the number of sentences in each poem text
poemsdf['num_sentences']= poemsdf['sentences'].map(lambda x: len(x))

In [43]:
?eh.abstraction_score()

In [44]:
#create feature variables, one a regular abstraction score, the other using lesk algorithm for basic word sense
#disambiguation
#Each calculation takes about 10 minutes to run, the following lines of code should take about 20 minutes

#start a timer clock for fun
t0= time.clock()
poemsdf['abstraction_score'] = poemsdf['sentences'].apply(lambda x: eh.abstraction_score(x))
print time.clock() - t0, "seconds process time"

t0= time.clock()
poemsdf['lesk_abs_score'] = poemsdf['sentences'].apply(lambda x: eh.abstraction_score(x, True))
print time.clock() - t0, "seconds process time"

In [45]:
?eh.pronoun_score()

In [46]:
#this might take a while
t0= time.clock()
poemsdf['pronoun_score'] = poemsdf['sentences'].map(lambda x: eh.pronoun_score(x))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['pronoun_score'] = poemsdf['pronoun_score']/poemsdf['num_sentences']
print time.clock() - t0, "seconds process time"

In [48]:
poemsdf.columns
# poemdf['bio_sents'] = poemdf['poet_bio'].map(lambda x: sent_tokenize(u''.join(x)))
# poemdf.columns
#looking around to see if anything can be done with the poet bios
# text = poemdf.bio_sents[15]
# text[0]

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',          u'numlines',
              u'numstanzas',    u'avgline_stanza',         u'wordcount',
             u'wordcount_d',               u'ttr',             u'yrpub',
               u'sentences',     u'num_sentences', u'abstraction_score',
          u'lesk_abs_score',     u'pronoun_score'],
      dtype='object')

In [50]:
# Look at relationship between words in title and words in text?

#abstraction score of title using lesk
#assume each title is a sentence/phrase
poemsdf['title_clean'] = poemsdf['poem_title'].map(lambda x: u''.join(x).replace('[','').replace(']',''))
#just checking if anything can be done
poemsdf['title_lesk_abs'] = poemsdf['title_clean'].map(lambda x: eh.abstraction_score(x, True))

In [67]:
poemsdf.columns

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',          u'numlines',
              u'numstanzas',    u'avgline_stanza',         u'wordcount',
             u'wordcount_d',               u'ttr',             u'yrpub',
               u'sentences',     u'num_sentences', u'abstraction_score',
          u'lesk_abs_score',     u'pronoun_score',       u'title_clean',
          u'title_lesk_abs', u'conjunction_ratio'],
      dtype='object')

##CALCULATE PHRASE COMPLEXITY

What is a noun phrase?
e.g. "head of lettuce", "creaky door", "Fred's bagel express"; noun is the head or main word

What is an adjective phrase?
An adjective phrase (or adjectival phrase) is a phrase whose head word is an adjective, e.g. fond of steak, very happy, quite upset about it, etc.

Adverb Adjective
Adjective Preposition Noun
Adverb Adjective Preposition Noun

What is a verb phrase?
A verb phrase is a syntactic unit composed of at least one verb and its dependents—objects, complements and other modifiers—but not always including the subject.

verb noun
verb verb noun

In [66]:
#CREATE FEATURE Conjunction_ratio
t0= time.clock()
poemsdf['conjunction_ratio'] = poemsdf['sentences'].apply(lambda x: eh.conjunction_ratio(x))
print time.clock() - t0, "seconds process time"

In [68]:
# ?eh.naive_enj_score()

t0= time.clock()
poemsdf['enj_score'] = poemsdf['sentences'].apply(lambda x: eh.naive_enj_score(x))
print time.clock() - t0, "seconds process time cacl enjambment score"

#CREATE FEATURES Noun, Adjective, Verb Phrasal Ratios - about 20-25 min each to run

t0= time.clock()
poemsdf['nps_ratio'] = poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'nps'))
print time.clock() - t0, "seconds process time calc p_ratio"
t0= time.clock()
poemsdf['vps_ratio'] = poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'vps'))
print time.clock() - t0, "seconds process time calc p_ratio"
t0= time.clock()
poemsdf['aps_ratio']= poemsdf['sentences'].apply(lambda x: eh.p_ratio(x,'aps'))
print time.clock() - t0, "seconds process time calc p_ratio"

#CREATE FEATURES Noun, Adjective, Verb Phrasal Complexity Scores - about 10-15 min to run

t0= time.clock()
poemsdf['avg_nps_cscore'] = poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'nps'))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['avg_vps_cscore'] = poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'vps'))
print time.clock() - t0, "seconds process time"
t0= time.clock()
poemsdf['avg_aps_cscore']= poemsdf['sentences'].apply(lambda x: eh.average_phrasal_complexity(x,'aps'))
print time.clock() - t0, "seconds process time"

In [4]:
# poemsdf = pd.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')
poemsdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')

In [52]:
extradf = poemsdf[['poem_link', 'sentences']]

In [65]:
t0= time.clock()
extradf['phrasalfreq'] = extradf['sentences'].apply(lambda x: eh.phrasal_freq(x))
print time.clock() - t0, "seconds process time"

In [66]:
#1464
len(extradf)

1464

In [94]:
extradf2 = eh.unpack_dictionary(extradf, 'phrasalfreq', fillna = 0.0)

In [100]:
# extradf2.dtypes
# poemsdf = 
poemsdf = pd.merge(poemsdf, extradf2, how='inner', on=['poem_link'])

In [103]:
poemsdf.drop('sentences_y', axis = 1, inplace = True)
poemsdf.columns

Index([        u'poem_link',    u'poet_poems_url',        u'poem_yrpub',
              u'poem_title',         u'poem_text',          u'numlines',
              u'numstanzas',    u'avgline_stanza',         u'wordcount',
             u'wordcount_d',               u'ttr',             u'yrpub',
             u'sentences_x',     u'num_sentences', u'abstraction_score',
          u'lesk_abs_score',     u'pronoun_score',       u'title_clean',
          u'title_lesk_abs', u'conjunction_ratio',         u'enj_score',
               u'nps_ratio',         u'vps_ratio',         u'aps_ratio',
          u'avg_nps_cscore',    u'avg_vps_cscore',    u'avg_aps_cscore',
             u'10_w_nps_fr',       u'11_w_nps_fr',       u'12_w_nps_fr',
             u'13_w_nps_fr',       u'14_w_nps_fr',       u'15_w_nps_fr',
             u'16_w_nps_fr',        u'1_w_nps_fr',       u'29_w_nps_fr',
              u'2_w_nps_fr',       u'34_w_nps_fr',        u'3_w_nps_fr',
              u'4_w_nps_fr',        u'5_w_nps_fr', 

In [112]:
# 29, 34, and 78 are outliers lets take look at the poems in there and drop them
poemsdf = poemsdf.loc[poemsdf['78_w_nps_fr']==0] # only one poem has this and its because nltk can't POS correctly
poemsdf = poemsdf.loc[poemsdf['34_w_nps_fr']==0] # this is an excerpt of a poem, can drop
poemsdf = poemsdf.loc[poemsdf['29_w_nps_fr']==0] # nltk pos def fails on this one, can drop

In [114]:
poemsdf.drop(['78_w_nps_fr','34_w_nps_fr','29_w_nps_fr'], axis = 1, inplace = True)

In [116]:
poemsdf.columns
poemsdf = pd.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/poemsdf.pkl')

#Combine cleaned poet dataframe and poem data frame based on the foreign key identifier, poet_poems_url
poetdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/poetdf.pkl')
resultdf = pd.merge(poetdf, poemsdf, how='inner', on=['poet_poems_url'])

In [2]:
# resultdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/resultdf.pkl')
resultdf = pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/resultdf.pkl')

Split tagged poems into groups

1.  Experimental 
    Surrealism – dreamy, used methods like automatic writing (free write), self induced hallucinations, “close observation of the dream state and free play of thought”
    Symbolists – “structures and conceits are built upon grand, illogical, intuitive associations”
    New York School – influenced by surrealism & abstract expressionist art, urban
    Concrete Poetry – visual poems, structure/shape of the poem important
    Language Poetry – language dictates meaning, reader involvement
    Imagism – clarity of expression through precise visual images
    Jazz Poetry – Jazz as poetic inspiration
    Black Mountain – projective verse, remove subjective, project onto the reader
    Conceptual poetry – uncreative, appropriation, the process as poetry

2.  Contemporary, Objectivsts

3.  Modern
    Modernism
    Beat – against conformity, against tradition
    SanFrancisco Renaissance – responding to wars and cultural environment
    Harlem Renaissance – African American identity
    Black arts – civil rights movement, explored African American cultural/historical
    Darkroom Collective – poets of color
    Confessional Poetry – personal, subjective
    Slam/Spokenword – personal subjective

4.  Traditional-Pure
    Formalism, 
    New Formalism – return to traditional forms, meter, rhyme, stanzaic forms
    FiresidePoet – conventional forms, non experimental, attention to rhymes, metrical cadence

In [3]:
resultdf['primary_tag'].value_counts()

# ##lets label by contemporary or not to make this easy on ourselves
resultdf['label'] = resultdf['primary_tag'].map(lambda x: eh.labler(x))

resultdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/labeleddf.pkl')

In [4]:
resultdf= pd.read_pickle('/Users/huiru/GADataScience/FinalProject/Data/labeleddf.pkl')
?eh.labler()

In [5]:
##alternatively, put string labels on as a separate df/pickle file
strlabeledf = resultdf
strlabeledf['strlabel'] = strlabeledf['primary_tag'].map(lambda x: eh.labler(x,True))

In [6]:
strlabeledf = strlabeledf.query('strlabel != ""')

In [7]:
strlabeledf.strlabel.value_counts()

Contemp    736
Exp        370
Modern     270
Trad        64
dtype: int64

In [8]:
# resultdf.label.value_counts()
resultdf = resultdf.query('primary_tag != "MetaphysicalPoet"')
resultdf.label.value_counts()

1    736
0    704
dtype: int64

In [9]:
stringcols = ['poet_name','poet_url','poet_poems_url','poet_bio','city','state','country','primary_tag','tags','tag2','tag3','tag4','poem_yrpub','poem_link', 'poem_title', 'title_clean','poem_text', 'sentences_x']
resultdf.dtypes[20:]
#EXCLUDE wordcount, wordcount_d
numerical = ['yr_born', 'yr_died','yrpub', 'numlines','numstanzas' ,'avgline_stanza','enj_score','ttr',
             'abstraction_score', 'lesk_abs_score', 'pronoun_score', 'title_lesk_abs', 'conjunction_ratio',
             'nps_ratio','vps_ratio','aps_ratio','avg_nps_cscore','avg_vps_cscore','avg_aps_cscore',
             '1_w_nps_fr','2_w_nps_fr','3_w_nps_fr','4_w_nps_fr','5_w_nps_fr','6_w_nps_fr','7_w_nps_fr',
             '8_w_nps_fr','9_w_nps_fr','10_w_nps_fr','11_w_nps_fr','12_w_nps_fr','13_w_nps_fr', '14_w_nps_fr',
             '15_w_nps_fr','16_w_nps_fr'
            ]
numerical.append('label')

In [10]:
df = resultdf[numerical]

In [11]:
strl = ['yr_born', 'yr_died','yrpub', 'numlines','numstanzas' ,'avgline_stanza','enj_score','ttr',
             'abstraction_score', 'lesk_abs_score', 'pronoun_score', 'title_lesk_abs', 'conjunction_ratio',
             'nps_ratio','vps_ratio','aps_ratio','avg_nps_cscore','avg_vps_cscore','avg_aps_cscore',
             '1_w_nps_fr','2_w_nps_fr','3_w_nps_fr','4_w_nps_fr','5_w_nps_fr','6_w_nps_fr','7_w_nps_fr',
             '8_w_nps_fr','9_w_nps_fr','10_w_nps_fr','11_w_nps_fr','12_w_nps_fr','13_w_nps_fr', '14_w_nps_fr',
             '15_w_nps_fr','16_w_nps_fr','strlabel','label'
            ]

strdf = strlabeledf[strl]

In [12]:
inds = pd.isnull(df).any(1).nonzero()[0]
strinds = pd.isnull(strdf).any(1).nonzero()[0]
df.ix[inds]
strdf.ix[strinds]

,yr_born,yr_died,yrpub,numlines,numstanzas,avgline_stanza,enj_score,ttr,abstraction_score,lesk_abs_score,...,9_w_nps_fr,10_w_nps_fr,11_w_nps_fr,12_w_nps_fr,13_w_nps_fr,14_w_nps_fr,15_w_nps_fr,16_w_nps_fr,strlabel,label
21,1903,1970,2002,9,3,3.000000,9.000000,1.000000,1.190476,1.000000,...,0,0,0,0,0,0,0,0,Contemp,1
40,1908,1984,2002,5,2,2.500000,4.333333,0.944444,1.763889,1.416667,...,0,0,0,0,0,0,0,0,Contemp,1
46,0,0,2010,5,1,5.000000,5.000000,0.941176,1.326190,1.214286,...,0,0,0,0,0,0,0,0,Exp,0
47,0,0,2010,6,2,3.000000,7.000000,0.714286,1.314123,1.541667,...,0,0,0,0,0,0,0,0,Exp,0
48,0,0,2010,5,1,5.000000,5.000000,0.933333,1.227778,1.300000,...,0,0,0,0,0,0,0,0,Exp,0
50,0,0,2010,5,1,5.000000,5.000000,0.863636,1.354167,1.333333,...,0,0,0,0,0,0,0,0,Exp,0
67,1938,0,1974,4,2,2.000000,2.000000,0.866667,1.116071,0.687500,...,0,0,0,0,0,0,0,0,Exp,0
120,1942,0,1990,14,1,14.000000,21.500000,0.071429,1.250000,1.250000,...,0,0,0,0,0,0,0,0,Exp,0
144,0,0,2004,3,1,3.000000,3.000000,0.923077,1.500000,1.000000,...,0,0,0,0,0,0,0,0,Contemp,1
146,0,0,2004,3,1,3.000000,3.000000,1.000000,0.500000,0.500000,...,0,0,0,0,0,0,0,0,Contemp,1


In [13]:
df = df.fillna(df.mean())
strdf = strdf.fillna(strdf.mean())
# df

In [14]:
df.query('yrpub == 0')
df.groupby(['label'])['yrpub'].median()

label
0    1993
1    2009
Name: yrpub, dtype: int64

In [15]:
df.query('yrpub == 0')
# df.loc[df['yrpub'==0],]
df['yrpub'][290] = 1993
df['yrpub'][863] = 2009

In [16]:
df = df.query('yrpub >= 1900')

In [17]:
df.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/clean.pkl')

In [18]:
strdf.query('yrpub == 0')
strdf.groupby(['strlabel'])['yrpub'].median()
#modern = 1961
#contemp = 2009

strlabel
Contemp    2009
Exp        1998
Modern     1961
Trad       2001
Name: yrpub, dtype: int64

In [19]:
strdf.query('yrpub == 0')
strdf['yrpub'][290] = 1961
strdf['yrpub'][863] = 2009

In [20]:
strdf = strdf.query('yrpub >= 1900')

In [24]:
strdf.query('yrpub <= 1900')

,yr_born,yr_died,yrpub,numlines,numstanzas,avgline_stanza,enj_score,ttr,abstraction_score,lesk_abs_score,...,9_w_nps_fr,10_w_nps_fr,11_w_nps_fr,12_w_nps_fr,13_w_nps_fr,14_w_nps_fr,15_w_nps_fr,16_w_nps_fr,strlabel,label


In [25]:
strdf.to_pickle('/Users/huiru/GADataScience/FinalProject/Data/strlabeleddf.pkl')